## Info
based on notebook3

## Load packages and the model

In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

2025-09-24 15:22:59.438117: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-24 15:23:01.145380: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-24 15:23:01.145437: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-24 15:23:01.155302: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-24 15:23:02.226889: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-24 15:23:02.237623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Copy weights

In [2]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Look at the data

In [3]:
data_file['data']["Background_data"]["Test"]["DATA"].shape

(4511092, 19, 3)

In [4]:
X = data_file['data']["Background_data"]["Test"]["DATA"][:2048]
scale_data = data_file['data']['Normalisation']['norm_scale'][:].flatten()
offset_data =  data_file['data']['Normalisation']['norm_bias'][:].flatten()

In [14]:
offset_data.reshape(-1,3)

array([[ 36.,   0.,  72.],
       [ 14.,   0.,  73.],
       [  9.,   0.,  73.],
       [  8.,   0.,  72.],
       [  6.,   0.,  72.],
       [ 12.,  -0., 286.],
       [  6.,  -2., 285.],
       [  6.,  -2., 282.],
       [  6.,  -2., 286.],
       [ 58.,   0.,  72.],
       [ 44.,   0.,  72.],
       [ 35.,   0.,  72.],
       [ 28.,   0.,  72.],
       [ 22.,   0.,  72.],
       [ 21.,   0.,  72.],
       [ 20.,   0.,  73.],
       [ 18.,   0.,  73.],
       [ 18.,   0.,  72.],
       [ 17.,  -2.,  72.]])

In [15]:
scale_data.reshape(-1,3)

array([[ 16.,   1.,  64.],
       [  8.,  32.,  64.],
       [  2.,  32.,  64.],
       [  2.,  32.,  64.],
       [  1.,  32.,  64.],
       [  8., 128., 256.],
       [  2., 128., 256.],
       [  2., 128., 256.],
       [  2., 128., 256.],
       [ 32.,  64.,  64.],
       [ 16.,  64.,  64.],
       [ 16.,  64.,  64.],
       [ 16.,  64.,  64.],
       [ 16.,  64.,  64.],
       [ 16.,  64.,  64.],
       [ 16.,  64.,  64.],
       [ 16.,  64.,  64.],
       [ 16.,  64.,  64.],
       [  8.,  64.,  64.]])

In [46]:
X[204]

array([[-0.6875    ,  0.        ,  0.390625  ],
       [-0.625     ,  0.34375   ,  0.328125  ],
       [-1.        ,  1.71875   , -0.53125   ],
       [-0.5       ,  0.21875   , -0.546875  ],
       [-1.        ,  0.28125   ,  0.640625  ],
       [-1.        ,  1.5234375 ,  0.05859375],
       [-1.        ,  1.5546875 ,  0.0390625 ],
       [-3.        ,  0.015625  , -1.1015625 ],
       [-3.        ,  0.015625  , -1.1171875 ],
       [-0.5625    , -0.171875  , -0.328125  ],
       [-1.25      ,  0.203125  , -0.796875  ],
       [-2.1875    ,  0.        , -1.125     ],
       [-1.75      ,  0.        , -1.125     ],
       [-1.375     ,  0.        , -1.125     ],
       [-1.3125    ,  0.        , -1.125     ],
       [-1.25      ,  0.        , -1.140625  ],
       [-1.125     ,  0.        , -1.140625  ],
       [-1.125     ,  0.        , -1.125     ],
       [-2.125     ,  0.03125   , -1.125     ]])

In [34]:
# selected row
X[:,9]

array([[ 1.21875 ,  0.8125  , -0.109375],
       [-0.75    , -0.234375,  0.984375],
       [-0.5     , -0.015625, -0.328125],
       ...,
       [-0.4375  ,  0.8125  , -0.140625],
       [ 0.      , -1.40625 ,  0.734375],
       [ 0.28125 ,  0.578125, -1.078125]])

(57,)

In [ ]:
i = 8
(X[:,i]*scale_data.reshape(-1,3)[i] + offset_data.reshape(-1,3)[i])

array([[  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       ...,
       [  0.,   0.,   0.],
       [ 21., 207., 308.],
       [  0.,   0.,   0.]])